# 非線形最適化

目的関数が1変数関数のときの最急降下法を実装してみる。

In [31]:
def backTrack(f, df, x, d, alpha=10, rho=0.5, c1=0.01):
    while(f(x + alpha*d) > f(x) + c1 * df(x) * d * alpha):
        alpha = rho*alpha
    return alpha

def grad(f, x0, df=None, eps=0.001):
    '''
    Parameters
    ----------
    f: objective function
    x0: initial point
    df: derivative
    eps: epsilon
    
    Return
    ------
    x^*: minimizer
    '''
    while(abs(df(x0)) > eps):
        if df == None:
            df = lambda x0: (f(x0)+f(x0+0.01))/0.01
        d = -df(x0)
        alpha = backTrack(f, df, x0, d)
        x0 = x0 + alpha*d
    return x0

In [6]:
f = lambda x: x**2

In [3]:
f(2)

4

In [32]:
grad(f, 100, df=lambda x: 2*x)

-0.0003814697265625

In [33]:
import numpy as np
from scipy.optimize import minimize, newton

In [38]:
minimize(f,100)

      fun: 5.552074997367714e-17
 hess_inv: array([[ 0.50000004]])
      jac: array([ -1.28826571e-12])
  message: 'Optimization terminated successfully.'
     nfev: 21
      nit: 4
     njev: 7
   status: 0
  success: True
        x: array([ -7.45122473e-09])

## 2変数のも作ってみる

In [40]:
g = lambda x, y: x + 2*y

In [41]:
g(1,2)

5

In [63]:
def backTrack(f, df, x, d, alpha=10, rho=0.5, c1=0.01):
    while(f(x + alpha*d) > f(x) + c1 * np.dot(df(x), d) * alpha):
        alpha = rho*alpha
    return alpha

def grad(f, x0, df=None, eps=0.001):
    '''
    Parameters
    ----------
    f: objective function
    x0: initial point
    df: derivative
    eps: epsilon
    
    Return
    ------
    x^*: minimizer
    '''
    while(np.linalg.norm(df(x0)) > eps):
        d = -df(x0)
        alpha = backTrack(f, df, x0, d)
        x0 = x0 + alpha*d
    return x0

In [64]:
import numpy as np
l = 10
def f(z):
    x = z[0]
    y = z[1]
    val = - x - y - l * (np.log((-1/3)*x-y+1) + np.log(-3*x-y+3) + np.log(x) + np.log(y) )
    return val

def df(z):
    x = z[0]
    y = z[1]
    val = [ (1 + l * (-1/(-x-3*y+3) - 3/(-3*x-y+3) + 1/x)), (1 + l * (-3/-x-3*y+3 - 1/(-3*x-y+3) + 1/y)) ]
    return np.array(val)

In [54]:
f([0.5,0.5])

23.849066497880003

In [55]:
df([0.5,0.5])

[-19.0, 86.0]

In [65]:
x0 = np.array([0.5, 0.5])
grad(f, x0, df)

/Users/KYO/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in log
/Users/KYO/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/ipykernel/__main__.py:12: RuntimeWarning: overflow encountered in double_scalars


KeyboardInterrupt: 

In [58]:
a = np.array([1,1])

In [59]:
-a

array([-1, -1])